In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, Dataset, random_split
import matplotlib.pyplot as plt
import os
from PIL import Image


In [2]:

# Custom dataset for labeled images
class LabeledImageDataset(Dataset):
    def __init__(self, image_folder, labels_file, transform=None):
        self.image_folder = image_folder
        self.labels = np.load(labels_file)  # Load labels from numpy array
        self.image_files = sorted(os.listdir(image_folder))  # Ensure sorted order
        self.transform = transform
    
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.image_folder, self.image_files[idx])
        image = Image.open(img_path).convert("L")  # Convert to grayscale
        label = torch.tensor(0 if self.labels[idx] == 0 else 1, dtype=torch.long)
        if self.transform:
            image = self.transform(image)
        return image, label


def get_dataloader(image_folder, labels_file, batch_size=4):
    transform = transforms.Compose([transforms.Grayscale(), transforms.ToTensor()])
    dataset = LabeledImageDataset(image_folder, labels_file, transform=transform)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return dataloader


# Main execution
image_folder = "labeled_images"
labels_file = "labels.npy"
# dataloader = get_dataloader(image_folder, labels_file)



dataset = LabeledImageDataset(image_folder, labels_file, transform=transforms.Compose([transforms.Grayscale(), transforms.ToTensor()]))

# Calculate the sizes for training and test sets
test_size = int(0.12 * len(dataset))
train_size = len(dataset) - test_size

# Split the dataset
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create DataLoaders for training and test sets
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

# Print the number of batches in train and test loaders
print(f"Number of training batches: {len(train_loader)}")
print(f"Number of test batches: {len(test_loader)}")
    



Number of training batches: 22
Number of test batches: 3


In [3]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1,1))
        )
        self.fc = nn.Linear(64, 2)  # Two classes: 0 and 1
    
    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

class DiffusionClassifier:
    def __init__(self):
        self.model = Classifier()
        self.optimizer = optim.Adam(self.model.parameters(), lr=1e-3)
        self.criterion = nn.CrossEntropyLoss()
    
    def train(self, dataloader, epochs=5):
        for epoch in range(epochs):
            for images, labels in dataloader:
                images, labels = images, labels
                outputs = self.model(images)
                loss = self.criterion(outputs, labels)
                
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
            print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")
    
    def predict(self, images):
        self.model.eval()
        with torch.no_grad():
            images = images.cuda()
            outputs = self.model(images)
            predictions = torch.argmax(outputs, dim=1).cpu().numpy()
        return predictions

# Data preparation for labeled grayscale PNG images

# train model
diffusion_classifier = DiffusionClassifier()
diffusion_classifier.train(train_loader)


Epoch 1, Loss: 0.6915
Epoch 2, Loss: 0.9326
Epoch 3, Loss: 0.6039
Epoch 4, Loss: 0.4501
Epoch 5, Loss: 1.1831
